<a href="https://colab.research.google.com/github/41371131h-chi/114-1-/blob/main/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, time, uuid, re, json, datetime
# 標準且清晰的匯入方式
import datetime as dt_module # 將整個 datetime 模組別名為 dt_module
from datetime import datetime as dt # 將 datetime 類別別名為 dt，用於呼叫 now()
from datetime import timedelta     # 將 timedelta 類別單獨匯入
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup
import pytz
from IPython.display import display, Markdown
from gspread_dataframe import set_with_dataframe, get_as_dataframe

import google.generativeai as genai

In [2]:
# Google Auth & Sheets
from google.colab import auth
import gspread
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default
from google.colab import auth
auth.authenticate_user()

In [3]:
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
from google.colab import files
import os

In [4]:
print("請上傳您的 PDF 課表檔案...")
# 顯示上傳按鈕
uploaded = files.upload()

pdf_filename = None
if not uploaded:
  print("\n⚠️ 您沒有上傳任何檔案。")
else:
  # 取得上傳檔案的名稱
  pdf_filename = next(iter(uploaded))
  print(f"\n✅ 成功上傳檔案: '{pdf_filename}'")

  # 檢查是否為 PDF (簡易檢查)
  if not pdf_filename.lower().endswith('.pdf'):
    print(f"⚠️ 警告：您上傳的檔案 '{pdf_filename}' 不是 PDF 檔。")
%store pdf_filename

請上傳您的 PDF 課表檔案...


Saving export.pdf to export (3).pdf

✅ 成功上傳檔案: 'export (3).pdf'
Stored 'pdf_filename' (str)


In [5]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
# 警告：此處使用 'HW3' 作為 Secret 名稱，請確保您已設置
api_key = userdata.get('HW3')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

In [6]:
# 讀取上一個儲存格存的檔案名稱
try:
    %store -r pdf_filename

    if pdf_filename is None or not os.path.exists(pdf_filename):
        print(f"❌ 找不到檔案 '{pdf_filename}'，請重新執行上一個儲存格上傳。")
    else:
        print(f"準備處理檔案: '{pdf_filename}'")

        # 1. 上傳檔案到 Gemini File API
        print("正在上傳 PDF 至 Gemini File API...")
        gemini_file = genai.upload_file(path=pdf_filename,display_name="114-1 課程表 PDF")
        print(f"✅ 檔案上傳成功！(File URI: {gemini_file.uri})")

        # 2. 準備 Prompt
        prompt_text = f"""
        你是一位 helpful AI assistant。
        請分析我上傳的 PDF 課表檔案 (檔案名稱: {pdf_filename})。

        這份 PDF 課表是「國立臺灣師範大學114學年度第1學期課表」。
        授課教師是「蔡芸琤」。

        請幫我將這份課表，轉換成每周的紀錄。
        學期開學日是 2025/9/1，學期結束日是 2025/12/19。

        你的輸出**必須**是 CSV 格式，並且**只**輸出 CSV 內容 (包含標頭)，
        不要有任何其他說明文字 (例如 "好的，這是..." 或 "```csv" 標記)。

        CSV 格式的欄位必須如下：
        "日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"

        - 「攜帶品」、「先讀章節」、「備註」這三欄因為 PDF 中沒有，請保持空白。
        - 「日期」範圍請涵蓋 2025/9/1 到 2025/12/19 之間所有有課的日期。
        - 請仔細解析 PDF 中的表格，找出所有課程、時間和地點。
        """

        # 3. 呼叫 Gemini 1.5 Pro 模型 (使用 Pro 處理 PDF 結構化效果更好)
        print("正在向 Gemini 2.5 Pro 提出請求 (這可能需要一點時間)...")
        model = genai.GenerativeModel('gemini-2.5-pro')

        # 發送包含「Prompt文字」和「上傳的檔案」的請求
        response = model.generate_content([prompt_text, gemini_file])

        # 4. 刪除在 Gemini 雲端上的檔案 (避免佔用空間)
        print("正在從 Gemini 雲端刪除暫存檔案...")
        genai.delete_file(gemini_file.name)
        print("✅ 暫存檔案已刪除。")

        # 5. 顯示結果並儲存
        csv_output = response.text.strip().replace("```csv\n", "").replace("```", "")
        print("\n--- 🤖 Gemini 產生的 CSV 內容 ---")
        print(csv_output)
        print("---------------------------------")

        # 6. 將 Gemini 產生的 CSV 內容存成一個檔案
        with open("gemini_schedule.csv", "w", encoding='utf-8') as f:
            f.write(csv_output)

        print("\n✅ 已將 Gemini 產生的結果儲存為 'gemini_schedule.csv'")

except NameError:
    print("❌ 找不到 'pdf_filename'。請先執行上一個儲存格來上傳檔案。")
except Exception as e:
    print(f"❌ 執行時發生錯誤：{e}")
    # 如果發生錯誤，也嘗試刪除已上傳的檔案
    try:
        if 'gemini_file' in locals():
            genai.delete_file(gemini_file.name)
            print("已清理遠端暫存檔案。")
    except Exception as del_e:
        print(f"清理檔案時發生錯誤: {del_e}")

準備處理檔案: 'export (3).pdf'
正在上傳 PDF 至 Gemini File API...
✅ 檔案上傳成功！(File URI: https://generativelanguage.googleapis.com/v1beta/files/swcnxa1gh1t5)
正在向 Gemini 2.5 Pro 提出請求 (這可能需要一點時間)...
正在從 Gemini 雲端刪除暫存檔案...
✅ 暫存檔案已刪除。

--- 🤖 Gemini 產生的 CSV 內容 ---
"日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"
"2025/9/1","星期一","機械製造","08:10","12:10","科技系TC504教室","","",""
"2025/9/1","星期一","教育哲學(教)","13:20","15:10","教310","","",""
"2025/9/1","星期一","班級經營(教)","15:30","17:20","誠107","","",""
"2025/9/2","星期二","木工製造","08:10","12:10","科技系TC501教室","","",""
"2025/9/2","星期二","色彩學","13:20","16:20","科技系TB213教室","","",""
"2025/9/3","星期三","3D列印與創新應用","10:20","12:10","誠102","","",""
"2025/9/4","星期四","程式語言","09:10","12:10","科技系TB311教室","","",""
"2025/9/4","星期四","經典與情緒素養","13:20","15:10","綜210展覽廳","","",""
"2025/9/4","星期四","機構設計","17:30","20:25","科技系TC504教室","","",""
"2025/9/5","星期五","電腦輔助製圖","14:20","17:20","科技系TA509教室","","",""
"2025/9/8","星期一","機械製造","08:10","12:10","科技系TC504教室","","",""
"2025/9/8","星期一","教育哲學

In [43]:
# --- 1. 定義您的 Google Sheet 資訊 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing"
WORKSHEET_NAME = "HW6"
TIMEZONE = "Asia/Taipei"
TW_TZ = pytz.timezone(TIMEZONE)

In [44]:
# --- 2. 讀取 Gemini 產生的 CSV 檔案 ---
try:
    df_to_upload = pd.read_csv("gemini_schedule.csv")
    print(f"✅ 成功讀取 'gemini_schedule.csv'，共 {len(df_to_upload)} 筆資料。")

    # --- 3. 準備 gspread ---
    df_to_upload = df_to_upload.fillna('')

    # --- 4. 開啟 Google Sheet 和指定的分頁 ---
    print(f"正在開啟 Google Sheet (URL: {SHEET_URL})...")
    gsheets = gc.open_by_url(SHEET_URL)

    print(f"正在存取分頁: '{WORKSHEET_NAME}'...")
    try:
        # 嘗試取得分頁，如果不存在就建立一個新的
        worksheet = gsheets.worksheet(WORKSHEET_NAME)
    except gspread.exceptions.WorksheetNotFound:
        print(f"找不到分頁 '{WORKSHEET_NAME}'，將自動建立新分頁...")
        worksheet = gsheets.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

    # --- 5. 將 DataFrame 寫入工作表 ---
    print(f"正在將 {len(df_to_upload)} 筆資料寫入 '{WORKSHEET_NAME}' 分頁...")
    # set_with_dataframe 會自動處理清空與標頭寫入
    set_with_dataframe(worksheet, df_to_upload)

    print(f"\n✅ 成功將 'gemini_schedule.csv' 的內容寫入至 '{WORKSHEET_NAME}' 分頁！")

except FileNotFoundError:
    print("❌ 錯誤：找不到 'gemini_schedule.csv' 檔案。")
    print("請確認您執行 Gemini API 的儲存格已成功產生這個檔案。")
except pd.errors.EmptyDataError:
    print("❌ 錯誤：'gemini_schedule.csv' 是空的，沒有資料可以上傳。")
except Exception as e:
    print(f"❌ 執行時發生錯誤：{e}")

✅ 成功讀取 'gemini_schedule.csv'，共 160 筆資料。
正在開啟 Google Sheet (URL: https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing)...
正在存取分頁: 'HW6'...
正在將 160 筆資料寫入 'HW6' 分頁...

✅ 成功將 'gemini_schedule.csv' 的內容寫入至 'HW6' 分頁！


In [45]:
# 從 gsheets 載入 sheets
try:
    gsheets = gc.open_by_url(SHEET_URL)
    sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
    df = pd.DataFrame(sheets[1:], columns=sheets[0])

    # 統一日期格式，為後續篩選做準備
    def ensure_datetime(s):
        try:
            if isinstance(s, str) and len(s) == 10 and s[4] == '/':
                return dt.strptime(s, '%Y/%m/%d').date()
            return pd.to_datetime(s).date()
        except Exception:
            return pd.NaT

    df["日期"] = df["日期"].apply(ensure_datetime)
    df = df.dropna(subset=["日期"])

except Exception as e:
    print(f"❌ 讀取 Google Sheet 時發生錯誤，請確認網路或 URL/分頁名稱是否正確: {e}")
    df = pd.DataFrame() # 設置空DataFrame以防止後續錯誤

# === 小工具函式 ===
week_map_tw = {"一":0,"二":1,"三":2,"四":3,"五":4,"六":5,"日":6}
week_map_en = {"Mon":0,"Tue":1,"Wed":1,"Thu":3,"Fri":4,"Sat":5,"Sun":6}

In [46]:
def normalize_weekday(w):
    if w in week_map_tw: return week_map_tw[w]
    if w in week_map_en: return week_map_en[w]
    raise ValueError("星期請用：一~日 或 Mon~Sun")

In [47]:
def week_monday(any_date):
    # 回到本週一
    return any_date - timedelta(days=any_date.weekday())

In [48]:
def date_range_this_week(today=None):
    # 取得「本週一 ~ 本週日」日期區間（以台北時區為準）
    # 這裡的 dt 是指向 datetime.datetime 類別 (來自 from datetime import datetime as dt)
    now = dt.now(TW_TZ).date() if today is None else today

    mon = week_monday(now)
    # 這裡的 timedelta 是指向單獨匯入的 timedelta 類別
    sun = mon + timedelta(days=6)
    return mon, sun

In [49]:
def summarize_courses(day_df):
    if day_df.empty: return "本日無課"
    parts = []
    for _, r in day_df.iterrows():
        parts.append(f"{r['時間(起)']}-{r['時間(迄)']} {r['課程名稱']}（{r['地點']}）")
    return "；".join(parts)

In [50]:
# === 擴充系統智能函式 A：生成行前提醒 (使用 Flash) ===

# 1. 簡化版 (Fallback) 必須先定義
def make_one_line_reminder_simple(rows):
    """簡單規則版本：只列出攜帶品和先讀章節"""
    if rows.empty: return "本日無課，不需準備。"
    items = []
    readings = []
    for _, r in rows.iterrows():
        if str(r.get("攜帶品", "")).strip(): items.append(str(r["攜帶品"]).strip())
        if str(r.get("先讀章節", "")).strip(): readings.append(str(r["先讀章節"]).strip())
    items_txt = "；".join(sorted(list(set(items)))) if items else "一般上課用品"
    read_txt = "；".join(sorted(list(set(readings)))) if readings else "無指定預習"
    return f"請攜帶：{items_txt}；預習：{read_txt}。"

# 2. 主函式現在可以直接呼叫它
def make_one_line_reminder(rows):
    """使用 Gemini 模型生成行前提醒"""
    if rows.empty:
        return make_one_line_reminder_simple(rows)

    # 從這裡開始執行 Gemini 邏輯
    print("🤖 正在使用 Gemini 2.5 Flash 擴充智能生成行前提醒...")

    course_list = "本日有以下課程：\n"
    for _, r in rows.iterrows():
        # 注意這裡的縮排，確保它在主函式內部
        detail = (
            f"  - 課程: {r['課程名稱']}, 時間: {r['時間(起)']}-{r['時間(迄)']}, 地點: {r['地點']}"
        )
        if r.get("攜帶品", "").strip(): detail += f", 攜帶品: {r['攜帶品'].strip()}"
        if r.get("先讀章節", "").strip(): detail += f", 預習: {r['先讀章節'].strip()}"
        if r.get("備註", "").strip(): detail += f", 備註: {r['備註'].strip()}"
        course_list += detail + "\n"

    prompt = f"""
    您是一位貼心的課程行前 AI 助理，專門為使用者準備上課提醒。
    請根據以下課程資訊，用**一句話**在繁體中文中生成一個簡潔、有條理的行前提醒。
    請綜合所有課程的「攜帶品」與「先讀章節」，以流暢的方式呈現。
    如果有多項指定，請用頓號（、）或分號（；）連接。
    如果沒有特別的攜帶品或先讀章節，請以輕鬆的方式（如「別忘了帶筆記本和筆」）作為替代提醒。
    請不要包含任何標頭、開頭語、或結尾標點符號 (如句號)，讓它像一個標題或口號。
    本日課程資訊：
    {course_list}
    """

    try:
        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content(prompt)
        return response.text.strip().replace("。", "").replace("！", "").replace("～", "")
    except Exception as e:
        print(f"⚠️ Gemini 模型生成提醒時發生錯誤，改用簡易規則。錯誤：{e}")
        return make_one_line_reminder_simple(rows)

In [51]:
# === 擴充系統智能函式 B：YouTube 影片推薦 (使用 Flash) ===
def recommend_youtube_videos(rows):
    """根據本週所有課程名稱，推薦相關 YouTube 影片或延伸閱讀"""
    if rows.empty:
        return "本日無課，無需推薦延伸資源。"

    print("🎬 正在使用 Gemini 2.5 Flash 推薦延伸學習資源...")

    unique_courses = rows['課程名稱'].unique().tolist()
    course_query = "、".join(unique_courses)

    prompt = f"""
    您是一位學術導師，請根據以下課程名稱，為學生推薦 1 到 3 個相關的 YouTube 影片教學連結或權威網站延伸閱讀。
    請以 Markdown 格式輸出，只列出「課程主題」和「連結 (URL)」，並用中文描述連結名稱。

    請推薦與這些課程相關的資源：{course_query}

    範例格式：
    ### 延伸學習資源推薦
    * [線性代數基礎：矩陣運算教學](https://youtube.com/link1)
    * [現代文學批判：結構主義導論](https://scholar.site/link2)
    """

    try:
        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"⚠️ 推薦資源時發生錯誤：{e}")
        return "推薦資源生成失敗。"

In [52]:
# === 擴充系統智能函式 C：個人學習計畫生成 (使用 Pro) ===
def generate_weekly_plan(goal_text, df_week):
    """根據使用者輸入的目標和本週課表，生成個人計畫"""
    print("📝 正在使用 Gemini 2.5 Pro 生成個人學習計畫...")

    course_summary = "本週課程列表：\n" + "\n".join([
        f"- 星期{r['星期']}: {r['課程名稱']} ({r['時間(起)']}-{r['時間(迄)']})"
        for _, r in df_week.iterrows()
    ])

    prompt = f"""
    您是一位高效時間管理顧問，請根據以下使用者輸入的「本週學習目標」和「本週課程表」，為他生成一個詳細且可執行的學習計畫。

    要求：
    1.  計畫必須涵蓋週一到週日。
    2.  需明確列出每天的學習任務和時間段。
    3.  請將課程表上的時間視為**已佔用**，並在課餘時間安排學習任務。
    4.  學習任務應具體、可量化，並與課程內容或使用者目標相關聯。
    5.  輸出格式必須是**Markdown 表格**。

    使用者輸入的本週學習目標：
    **{goal_text}**

    本週課程表：
    {course_summary}
    """

    try:
        model = genai.GenerativeModel('gemini-2.5-pro')
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"⚠️ 生成個人學習計畫時發生錯誤：{e}")
        return "個人學習計畫生成失敗。"

In [53]:
# === 新增功能 D & E：排程自動通知 & HTML 頁面生成 ===

def mock_schedule_tool(query, notification_text, days_of_week, time_of_day, every_n_weeks):
    """模擬呼叫 scheduler:schedule"""
    print("\n--- 🤖 模擬呼叫 scheduler:schedule ---")
    print(f"排程內容: {query}")
    print(f"排程時間: 每 {every_n_weeks} 週的 {days_of_week} {time_of_day[0]}")
    print("---------------------------------------")
    return f"已成功排程：每週一 {time_of_day[0]} 執行您的自動通知任務。"

In [54]:
def generate_weekly_html(df_week, selected_weekday, one_line, recommended_resources, this_mon, this_sun):
    """生成包含本週課程和 AI 提醒的 HTML 頁面"""
    # ... (HTML 內容保持不變，但參數從外面傳入)
    html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>我的本週課程摘要 ({this_mon} ~ {this_sun})</title>
    <meta charset="utf-8">
    <style>
        body {{ font-family: 'Noto Sans TC', sans-serif; background-color: #f4f7f6; color: #333; padding: 20px; }}
        .container {{ max-width: 800px; margin: auto; background: #fff; padding: 30px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }}
        h1, h2 {{ color: #007bff; border-bottom: 2px solid #eee; padding-bottom: 10px; }}
        table {{ width: 100%; border-collapse: collapse; margin-top: 20px; }}
        th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
        th {{ background-color: #007bff; color: white; }}
        .reminder {{ background-color: #fff3cd; color: #856404; padding: 15px; border: 1px solid #ffeeba; border-radius: 5px; margin-top: 20px; }}
    </style>
</head>
<body>
    <div class="container">
        <h1>🎓 我的本週課程摘要</h1>
        <h2>日期：{this_mon} ~ {this_sun}</h2>

        <div class="reminder">
            <h3>AI 行前提醒 (針對星期{selected_weekday}的課)</h3>
            <p><strong>{one_line}</strong></p>
        </div>

        <h2>本週詳細課表</h2>
        {df_week.to_html(index=False)}

        <h2>延伸學習資源 (針對星期{selected_weekday}的課)</h2>
        <div style="white-space: pre-wrap; font-size: 0.9em;">
            {recommended_resources}
        </div>
    </div>
</body>
</html>
"""
    filename = f"weekly_summary_{selected_weekday}_{this_mon}_{this_sun}.html"
    with open(filename, "w", encoding='utf-8') as f:
        f.write(html_content)
    return f"✅ 成功生成 HTML 檔案: <a href='{filename}' target='_blank'>{filename}</a>"

In [55]:
# === 核心邏輯函式：根據使用者選擇的星期執行所有分析和輸出 ===
def process_day(selected_weekday):
    """處理單一星期的課程分析、AI生成、回寫和輸出"""
    if df.empty:
        return ("❌ 錯誤：無法載入課程資料，請檢查 Google Sheet 設定。",
                "無法執行 AI 提醒。",
                "無法執行 AI 推薦。",
                "無法生成 HTML。",
                "無法更新 Google Sheet。")

    # 1. 數據過濾
    target_wd = normalize_weekday(selected_weekday)
    this_mon, this_sun = date_range_this_week()

    mask_week = (df["日期"] >= this_mon) & (df["日期"] <= this_sun)
    mask_wd   = df["日期"].apply(lambda d: d.weekday()==target_wd)
    today_df  = df[mask_week & mask_wd].copy().sort_values(["日期","時間(起)"])
    this_week_df = df[mask_week].copy().sort_values(["日期","時間(起)"])

    # 2. 執行擴充系統智能 A & B (行前提醒與推薦)
    one_line = make_one_line_reminder(today_df)
    recommended_resources = recommend_youtube_videos(today_df)

    # 3. 回寫「本週重點」分頁
    SHEET_WEEKLY = "本週重點"
    try:
        wk = gsheets.worksheet(SHEET_WEEKLY)
    except gspread.exceptions.WorksheetNotFound:
        wk = gsheets.add_worksheet(title=SHEET_WEEKLY, rows=100, cols=20)

    now_str = dt.now(TW_TZ).strftime("%Y-%m-%d %H:%M")

    row = {
        "週次起訖": f"{this_mon} ~ {this_sun}",
        "查詢星期": f"星期{selected_weekday}",
        "當日課程列表": summarize_courses(today_df),
        "行前提醒一句話": one_line,
        "更新時間": now_str
    }

    existing = pd.DataFrame(wk.get_all_records())
    outdf = pd.concat([existing, pd.DataFrame([row], columns=row.keys())], ignore_index=True) if not existing.empty else pd.DataFrame([row])

    wk.clear()
    set_with_dataframe(wk, outdf)
    sheet_status = f"✅ 已回寫到 **本週重點** 分頁（保留歷史列），針對星期{selected_weekday}的提醒。"

    # 4. 生成 HTML 頁面
    html_result = generate_weekly_html(this_week_df, selected_weekday, one_line, recommended_resources, this_mon, this_sun)

    # 5. 輸出結果 (Markdown 格式)
    course_markdown = (f"### 本週 ({this_mon} ~ {this_sun}) 星期{selected_weekday} 課程結果\n" +
                       today_df[["日期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"]].to_markdown(index=False))

    return course_markdown, one_line, recommended_resources, html_result, sheet_status

In [56]:
# === Gradio 編輯面板 (新增學習目標輸入) ===

def save_edited_reminder(edited_text, goal_input, selected_weekday):
    """讓使用者編輯後回存到 Google Sheet，並生成學習計畫"""

    # 重新執行一次數據篩選，確保 today_df, this_week_df 準確
    target_wd = normalize_weekday(selected_weekday)
    this_mon, this_sun = date_range_this_week()
    mask_week = (df["日期"] >= this_mon) & (df["日期"] <= this_sun)
    mask_wd   = df["日期"].apply(lambda d: d.weekday()==target_wd)
    today_df  = df[mask_week & mask_wd].copy().sort_values(["日期","時間(起)"])
    this_week_df = df[mask_week].copy().sort_values(["日期","時間(起)"])

    now_str = dt.now(TW_TZ).strftime("%Y-%m-%d %H:%M")
    SHEET_WEEKLY = "本週重點"

    # 1. 更新 Google Sheet (使用編輯後的文字)
    try:
        wk_recheck = gsheets.worksheet(SHEET_WEEKLY)
    except gspread.exceptions.WorksheetNotFound:
        wk_recheck = gsheets.add_worksheet(title=SHEET_WEEKLY, rows=100, cols=20)

    existing = pd.DataFrame(wk_recheck.get_all_records())
    row = {
        "週次起訖": f"{this_mon} ~ {this_sun}",
        "查詢星期": f"星期{selected_weekday}",
        "當日課程列表": summarize_courses(today_df),
        "行前提醒一句話": edited_text, # 使用編輯後的文字
        "更新時間": now_str
    }
    outdf = pd.concat([existing, pd.DataFrame([row], columns=row.keys())], ignore_index=True)
    wk_recheck.clear()
    set_with_dataframe(wk_recheck, outdf)

    gs_status = f"✅ 已更新至 Google Sheet『本週重點』分頁，目前共 {len(outdf)} 筆。"

    # 2. 生成個人學習計畫 (功能 C)
    if goal_input.strip():
        plan_markdown = generate_weekly_plan(goal_input, this_week_df)
        plan_status = "\n\n### 📝 您的個人學習計畫\n" + plan_markdown
    else:
        plan_status = "\n\n🤔 未輸入學習目標，跳過個人計畫生成。"

    return gs_status + plan_status

In [57]:
# === Gradio UI 佈局 ===
today_weekday_cn = ["一", "二", "三", "四", "五", "六", "日"][dt.now(pytz.timezone(TIMEZONE)).weekday()]

with gr.Blocks(title="課程助理與學習計畫編輯器") as demo:
    gr.Markdown("## 🗓️ 課程助理與學習計畫中心 (可選星期)")
    gr.Markdown("### 1. 選擇查詢的星期")

    # 星期選擇器和查詢按鈕
    day_selector = gr.Radio(
        ["一", "二", "三", "四", "五", "六", "日"],
        label="選擇您想查詢本週哪一天的課程與提醒",
        value=today_weekday_cn # 預設為今天
    )
    search_btn = gr.Button("🔍 執行查詢與 AI 分析")

    # 輸出結果區
    with gr.Tabs():
        with gr.TabItem("課程與 AI 分析結果"):
            course_table_output = gr.Markdown("點擊按鈕後，此處將顯示當日課表。")
            reminder_output_display = gr.Markdown("此處將顯示 AI 生成的行前提醒。")
            resource_output_display = gr.Markdown("此處將顯示 AI 推薦的延伸資源。")
            html_output_display = gr.Markdown("此處將顯示 HTML 檔案連結。")
            sheet_status_output = gr.Markdown("此處將顯示 Google Sheet 更新狀態。")

        with gr.TabItem("自動通知排程 (模擬)"):
            gr.Markdown("#### 📮 自動通知排程設定")
            # 由於排程是固定的，在 Gradio 中僅顯示一次模擬結果
            schedule_time = "08:00"
            schedule_day = "MONDAY"
            schedule_query = f"請生成並寄出本週 ({date_range_this_week()[0]} ~ {date_range_this_week()[1]}) 的課程摘要 PDF，並發送 LINE Notify 提醒。"
            schedule_notification = "您的本週課程摘要已備妥，點擊查看！"

            mock_schedule_result = mock_schedule_tool(
                query=schedule_query,
                notification_text=schedule_notification,
                days_of_week=[schedule_day],
                time_of_day=[schedule_time],
                every_n_weeks=1
            )
            gr.Markdown(f"> {mock_schedule_result}")

    gr.Markdown("---")
    gr.Markdown("### 2. 行前提醒編輯與個人學習計畫生成")

    with gr.Row():
        with gr.Column():
            reminder_box = gr.Textbox(label="行前提醒一句話 (可編輯)", lines=3)

        with gr.Column():
            goal_input = gr.Textbox(label="請輸入您的本週學習目標", lines=3)

    save_btn = gr.Button("💾 提交並生成計畫 (回存提醒 + 生成個人計畫)")
    final_status = gr.Markdown()

    # --- 互動邏輯 ---
    search_btn.click(
        process_day,
        inputs=[day_selector],
        outputs=[course_table_output, reminder_box, resource_output_display, html_output_display, sheet_status_output]
    )

    save_btn.click(
        save_edited_reminder,
        inputs=[reminder_box, goal_input, day_selector], # 傳入選擇的星期
        outputs=[final_status]
    )


--- 🤖 模擬呼叫 scheduler:schedule ---
排程內容: 請生成並寄出本週 (2025-11-10 ~ 2025-11-16) 的課程摘要 PDF，並發送 LINE Notify 提醒。
排程時間: 每 1 週的 ['MONDAY'] 08:00
---------------------------------------


In [58]:
demo.launch(share=True, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8df747ebf0c4072d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
